In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data=pd.read_csv("/kaggle/input/enarm-20012019/ENARM.csv")
df=pd.DataFrame(data)
df.head(10)

In [ ]:
df.rename({'estado': 'region', 'Escuela de Medicina': 'Medicine School', 'Sustentantes':'Sustainers',
           'Seleccionados':'selected','promedio':'average_marks','año':'year'}, axis=1, inplace=True)
df.head()

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
mean =[]
years = np.arange(2001,2020)
for yr in years:
    mean.append(df.average_marks[df.year.isin([yr])].mean())
average_year =pd.DataFrame(list(zip(years,mean)),columns=['Year', 'Average'])
average_year.fillna(0,inplace=True)
average_year.set_index('Year',inplace=True)
average_year.head()

In [ ]:
df['Variance']=np.zeros(df.shape[0])

for yr in years:
    df.loc[df.year.isin([yr]),'Variance']=df['average_marks'][df.year.isin([yr])] - average_year.at[yr,'Average']
        
df.head()

The 30 universities with the highest variance of the mean, these universities and the subsequent ones will be susceptible to fraud.

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
fig = px.scatter(df, x='year', y='average_marks', color='Medicine School',
             hover_name='Medicine School', hover_data=['region', 'average_marks', 'Sustainers','selected','Variance'])

fig.add_trace(go.Scatter(x = average_year.index, y = average_year['Average'], name='Average of the average grade'))

fig.update_layout(legend_orientation ='h',legend_y =-0.3,
                 title='Average grade per year and university')
fig.show()

You can see a university with a drastic change in the average grade but it may be due to the fact that the number of supporters is small.

You have to look for peaks that are out of the average of the universities.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
fig = px.scatter(df.sort_values('Variance',ascending=False)[0:100], x='year', y='Variance', color='Medicine School',
             hover_name='Medicine School', hover_data=['region', 'average_marks', 'Sustainers','selected','Variance'])

fig.update_layout(legend_orientation ='h',legend_y =-0.3,
                 title='100 Cases most susceptible to fraud')
fig.show()

A 'U' shaped curve can indicate the inflection point from which fraud began continuously, an 'A' shaped curve would indicate that fraud was stopped, a peak would be one-off fraud, and a constant line absence of fraud.

> College with potential fraud going on


In [ ]:
df.sort_values('Variance',ascending=False).head(30)